In [9]:
# --- Step 1: Install and Setup Environment (High Power Mode) ---

!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama

import subprocess
import time

print("⏳ Starting Ollama server...")
process = subprocess.Popen(["ollama", "serve"])
time.sleep(5)

# دانلود مدل 8 میلیاردی (Llama 3.1)
# این مدل حدود 4.7 گیگابایت حجم دارد و روی کولب عالی اجرا می‌شود
print("⬇️ Pulling Llama 3.1 (8B) model - The Smart Judge...")
!ollama pull llama3.1

print("✅ High-Performance Environment Ready!")

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
⏳ Starting Ollama server...
⬇️ Pulling Llama 3.1 (8B) model - The Smart Judge...

✅ High-Performance Environment Ready!


In [10]:
# --- Step 2: Create the Input Data File (grammar_test.jsonl) ---

import json

# Define the test cases provided by the professor
test_data = [
    {"id": "1", "task": "role_analysis", "question": "Nella frase 'Che bel tempo', qual è il ruolo grammaticale di 'Che'?", "answer_minerva": "In questa frase, 'Che' è un aggettivo esclamativo."},
    {"id": "2", "task": "role_analysis", "question": "Nella frase 'Il libro che ho letto', cos'è 'che'?", "answer_minerva": "Qui 'che' è un pronome relativo."},
    {"id": "3", "task": "role_analysis", "question": "Nella frase 'Spero che tu stia bene', cos'è 'che'?", "answer_minerva": "Qui 'che' è una congiunzione."},
    {"id": "4", "task": "grammar_check", "question": "Quale frase è corretta: A) 'Io sono andare a casa' o B) 'Io sono andato a casa'?", "answer_minerva": "La frase corretta è la B ('Io sono andato a casa'). La frase A usa l'infinito in modo errato."},
    {"id": "5", "task": "grammar_check", "question": "Analizza questa frase: 'Loro mangia la mela'. È corretta?", "answer_minerva": "No, non è corretta. Il verbo 'mangia' non concorda con il soggetto 'Loro'. La forma corretta è 'Loro mangiano la mela'."}
]

# Write data to a JSONL file
filename = "grammar_test.jsonl"
with open(filename, "w", encoding="utf-8") as f:
    for entry in test_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"✅ Data file '{filename}' created successfully with {len(test_data)} test cases.")

✅ Data file 'grammar_test.jsonl' created successfully with 5 test cases.


In [11]:
# --- Step 3: Run AI Evaluation with Llama 3.1 (8B) ---

import json
import csv
import ollama
import os

# --- Configuration ---
INPUT_JSONL = "grammar_test.jsonl"
OUT_CSV = "evaluations_grammar_PRO.csv"  # اسم فایل خروجی حرفه‌ای

# استفاده از مدل 8 میلیاردی برای داوری دقیق
JUDGE_MODELS = ["llama3.1"]

EVAL_PROMPT_IT = """Sei un professore universitario di linguistica italiana.
Valuta la correttezza e la pertinenza della risposta di Minerva.

IMPORTANTE: Rispondi ESCLUSIVAMENTE con un oggetto JSON valido.
Schema JSON:
{{
  "score_pertinenza": <intero 1-5>,
  "commento": "<analisi dettagliata e tecnica>"
}}

Domanda: {question}
Risposta: {answer}
"""

def read_jsonl(path):
    rows = []
    if not os.path.exists(path): return []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip(): rows.append(json.loads(line))
    return rows

def main():
    print("🚀 Starting Professional Evaluation with Llama 3.1 (8B)...")

    data = read_jsonl(INPUT_JSONL)
    if not data: return
    results = []

    for row in data:
        qid = row.get("id")
        question = row.get("question")
        answer = row.get("answer_minerva")

        print(f"\n📝 Judging Task ID: {qid}")
        row_result = {"id": qid, "task": row.get("task"), "question": question, "answer": answer}

        for model_name in JUDGE_MODELS:
            print(f"   ⚖️  Professor ({model_name}) is grading...")
            prompt = EVAL_PROMPT_IT.format(question=question, answer=answer)

            try:
                # استفاده از مدل 8 میلیاردی
                response = ollama.chat(
                    model=model_name,
                    messages=[{'role': 'user', 'content': prompt}],
                    format='json',
                    options={'temperature': 0.0}
                )

                score_data = json.loads(response['message']['content'])

                row_result[f"{model_name}_score"] = score_data.get('score_pertinenza')
                row_result[f"{model_name}_comment"] = score_data.get('commento')
                print(f"      ✅ Score: {score_data.get('score_pertinenza')}/5")

            except Exception as e:
                print(f"      ❌ Error: {e}")

        results.append(row_result)

    keys = results[0].keys() if results else []
    with open(OUT_CSV, "w", encoding="utf-8", newline="") as f:
        dict_writer = csv.DictWriter(f, keys)
        dict_writer.writeheader()
        dict_writer.writerows(results)

    print(f"\n🎉 Professional Evaluation Complete! Results saved to: {OUT_CSV}")

if __name__ == "__main__":
    main()

🚀 Starting Professional Evaluation with Llama 3.1 (8B)...

📝 Judging Task ID: 1
   ⚖️  Professor (llama3.1) is grading...
      ✅ Score: 2/5

📝 Judging Task ID: 2
   ⚖️  Professor (llama3.1) is grading...
      ✅ Score: 4/5

📝 Judging Task ID: 3
   ⚖️  Professor (llama3.1) is grading...
      ✅ Score: 2/5

📝 Judging Task ID: 4
   ⚖️  Professor (llama3.1) is grading...
      ✅ Score: 5/5

📝 Judging Task ID: 5
   ⚖️  Professor (llama3.1) is grading...
      ✅ Score: 5/5

🎉 Professional Evaluation Complete! Results saved to: evaluations_grammar_PRO.csv
